In [33]:
import pandas as pd
import nltk
import string
import collections
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk import bigrams

# Init Lemmatizer
lemmatizer = WordNetLemmatizer()
STORY = "story"
ASK_HN = "ask_hn"
SHOW_HN = "show_hn"
POLL = "poll"
DELTA = 0.5

def get_pos_tag(word) :
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def lemmatize_and_store(sentence, vocabulary, post_type_vocabulary) :
    lemmatized_words = []
    for word in nltk.word_tokenize(sentence.lower()) :
        if word not in string.punctuation :
            lemmatized_word = lemmatizer.lemmatize(word, get_pos_tag(word))
            lemmatized_words.append(lemmatized_word)
            add_to_vocabulary(lemmatized_word, vocabulary)
            add_to_vocabulary(lemmatized_word, post_type_vocabulary)
    return lemmatized_words


def add_to_vocabulary(word, vocabulary) :
    if word in vocabulary :
        vocabulary[word] += 1
    else :
        vocabulary[word] = 1


def get_lemmatized_words(sentence) :
    lemmatized_words = []
    for word in nltk.word_tokenize(sentence.lower()) :
        if word not in string.punctuation :
            lemmatized_word = lemmatizer.lemmatize(word, get_pos_tag(word))
            lemmatized_words.append(lemmatized_word)
    return lemmatized_words


def create_vocabulary(posts, vocabulary, post_type_vocabulary, bigram_vocabulary, post_type) :
    for index, post in posts.iterrows():
        cleaned_words = lemmatize_and_store(post.Title, vocabulary, post_type_vocabulary)
        string_bigrams = bigrams(cleaned_words)
        for gram in string_bigrams: 
            add_to_bigram_vocabulary(gram, bigram_vocabulary, post_type)


def add_to_bigram_vocabulary(word, bigram_vocabulary, post_type) :
    if word in bigram_vocabulary :
        bigram_vocabulary[word][0] += 1
        if post_type in bigram_vocabulary[word][1] :
            bigram_vocabulary[word][1][post_type] += 1
        else :
            bigram_vocabulary[word][1][post_type] = 1
    else :
        post_type_dict = dict()
        post_type_dict[post_type] = 1
        bigram_vocabulary[word] = [1, post_type_dict]


def calculate_bigram_prob(bigram_vocabulary, vocabulary, bigram_probability, delta):
    for gram in bigram_vocabulary:
        prob = (bigram_vocabulary[gram][0] + delta) / (vocabulary[gram[0]] + (len(vocabulary) * delta))
        bigram_probability[gram] = prob



csvdf = pd.read_csv('hn2018_2019.csv', delimiter=',', encoding='utf-8')

data_2018 = csvdf[(csvdf["Created At"] >= "2018-01-01 00:00:00") & (csvdf["Created At"] <= "2018-12-31 23:59:59")]

data_2018 = data_2018.groupby("Post Type")

story_posts = data_2018.get_group(STORY)
ask_posts = data_2018.get_group(ASK_HN)
show_posts = data_2018.get_group(SHOW_HN)
poll_posts = data_2018.get_group(POLL)

# show_posts

vocabulary = dict()
story_post_vocabulary = dict()
ask_post_vocabulary = dict()
show_post_vocabulary = dict()
poll_post_vocabulary = dict()
bigram_vocabulary = dict()
bigram_probability = dict()

# create_vocabulary(story_posts, vocabulary, story_post_vocabulary, bigram_vocabulary, STORY)
# create_vocabulary(ask_posts, vocabulary, ask_post_vocabulary, bigram_vocabulary, ASK_HN)
# create_vocabulary(show_posts, vocabulary, show_post_vocabulary, bigram_vocabulary, SHOW_HN)
create_vocabulary(poll_posts, vocabulary, poll_post_vocabulary, bigram_vocabulary, POLL)

# for gram in bigram_vocabulary:
#     print(gram, ": ", bigram_vocabulary[gram][0])

calculate_bigram_prob(bigram_vocabulary, vocabulary, bigram_probability, DELTA)

print("Done..")

bigram_probability = collections.OrderedDict(sorted(bigram_probability.items(), key=lambda kv:kv[1], reverse=True))
# for gram in bigram_probability:
#     print(gram, ": ", bigram_probability[gram])


# Task 1 Build the model



Done..


In [ ]:
# Lemmatize a Sentence with the appropriate POS tag
# sentence = """Following mice attacks MySQL 10% HN: on UAE ASK-HR Dr. Ph.D. sagar's $300 etc. caring farmers were marching to Delhi for better living conditions. 
# Delhi police on Tuesday fired water cannons and teargas shells at protesting farmers as they tried to 
# break barricades with their cars, automobiles and tractors."""

# print("\nOrignal Sentence: ")
# print(sentence)

# print("\nNew Sentence: ")
# newSentence = get_lemmatized_words(sentence)
# print(newSentence)
# print("\nPunctuation",string.punctuation)

# print("\n")
# string_bigrams = bigrams(newSentence)
# for gram in string_bigrams: 
#     print(gram)

# print("\nSplit:",newSentence)